In [110]:
import pandas as pd
import glob

# path to the folder containing csv files
path = "C:\\Users\\rafae\\OneDrive\\Área de Trabalho\\data"

# use glob to get a list of all csv files in the folder
all_files = glob.glob(path + "/*.csv")

# create an empty list to store the dataframes for each file
dfs = []

# loop through each file and read it using pandas read_csv
for filename in all_files:
    df = pd.read_csv(filename)
    dfs.append(df)

# concatenate all dataframes into one
all_data = pd.concat(dfs, ignore_index=True)

In [112]:
all_data.to_csv("data_analyst.csv",index=False)

In [113]:
#all_data.to_csv("data_analyst.csv",index=False)
all_data.shape[0]
all_data.drop_duplicates(subset='Job_ID', keep='first')
all_data.shape[0]

14636

In [114]:
#all_data[all_data['Description'].str.contains('salary')]
#Competitive salary
import re
regex_pattern = r'\$([\d,]+)\s*(?:-\s*\$([\d,]+))?'#r'\$([\d,]+)\s*(?:-|\bto\b)\s*\$([\d,]+)|\$([\d,]+)'
#
def extract_salary(description):
    match = re.search(regex_pattern, description)
    if match:
        salary_range = match.groups()
        if salary_range[1]: # salary is a range
            salary_min = float(salary_range[0].replace(',', ''))
            salary_max = float(salary_range[1].replace(',', ''))
            return (salary_min + salary_max) / 2
        else: # salary is a single value
            salary = float(salary_range[0].replace(',', ''))
            return salary
    else: # no salary information found in description
        return None

# apply extract_salary function to Description column
all_data['Description'].apply(extract_salary)

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
14631   NaN
14632   NaN
14633   NaN
14634   NaN
14635   NaN
Name: Description, Length: 14636, dtype: float64

In [115]:
def extract_salary(description):
    match = re.search(regex_pattern, description)
    if match:
        salary = match.group(1).replace(',', '')
        return float(salary)
    else:
        return None
    
all_data['Salary'] = all_data['Description'].apply(extract_salary)

In [116]:
all_data.value_counts('Place').head(20)

Place
Berlin, Berlin, Germany                 863
Seattle, WA                             826
London, England, United Kingdom         807
Barcelona, Catalonia, Spain             727
Boston, MA                              711
Toronto, Ontario, Canada                710
Vancouver, British Columbia, Canada     663
Madrid, Community of Madrid, Spain      551
London, England, United Kingdom         441
Seattle, WA                             274
Redmond, WA                             258
San Francisco, CA                       207
Canada, Moyen-Chari, Chad               205
Toronto, Ontario, Canada                198
Vancouver, British Columbia, Canada     190
Brussels, Brussels Region, Belgium      188
Berlin, Germany                         175
Mississauga, Ontario, Canada            174
Boston, MA                              170
Bellevue, WA                            154
dtype: int64

In [117]:
regex_pattern = r'(\b\w+[.]?(?=\s*,|\s*$))'
def extract_country(location):
    match = re.search(regex_pattern, location)
    if match:
        return match.group(1)
    else:
        return None

# apply the function to the 'Location' column of the DataFrame
all_data['City'] = all_data['Place'].apply(extract_country)

In [59]:
keywords_lower = ['python','sql','r', 'tableau', 'power bi','azure','google cloud','databricks','git','aws','nosql','spark','docker','gcp','looker','etl','snowflake','redshift','excel','sas','javascript']
#all_data['Description'].str.lower().str.contains(keywords_lower).astype(int)
# iterate over the keywords and create a new column for each keyword
for keyword in keywords_lower:
    all_data[keyword] = all_data['Description'].str.lower().str.contains(keyword).astype(int)

In [118]:
all_data['Description'].str.lower().str.contains('python')

0         True
1         True
2         True
3        False
4        False
         ...  
14631     True
14632     True
14633    False
14634     True
14635     True
Name: Description, Length: 14636, dtype: bool

In [119]:
all_data.head()

,Job_ID,Link,Apply Link,Title,Company,Place,Description,HTML,Date,Salary,City
0,3449267566,https://www.linkedin.com/jobs/view/data-analys...,NaN,Data Analyst,Near,"Pasadena, CA","Description\n\n\n\n\nYou will be joining Near,...","<div class=""description__text description__tex...",2023-01-23,NaN,Pasadena
1,3424527214,https://www.linkedin.com/jobs/view/data-analys...,NaN,Data Analyst,RennerBrown,"New York, United States","*NO 3RD PARTIES*\n\nThe Analyst, Business Oper...","<div class=""description__text description__tex...",2023-01-03,NaN,York
2,3422151601,https://www.linkedin.com/jobs/view/data-analys...,NaN,Data Analyst,PointsBet,"Denver, CO",Data Analyst\n\n\n\n\nPointsBet (ASX: PBH) is ...,"<div class=""description__text description__tex...",2023-01-05,NaN,Denver
3,3421351929,https://www.linkedin.com/jobs/view/junior-data...,NaN,Junior Data Analyst - 100% Remote,Visionaire Partners,"Norcross, GA",Data Analyst - Operations (Live Anywhere - mus...,"<div class=""description__text description__tex...",2023-01-05,NaN,Norcross
4,3451447956,https://www.linkedin.com/jobs/view/data-analys...,NaN,Data Analyst,Spruce,"Austin, TX","Who We Are\n\nAt Spruce, our mission is to cha...","<div class=""description__text description__tex...",2023-01-23,NaN,Austin


In [120]:
all_text = all_data['Description'].str.cat(sep=' ')
# split string into list of words and count occurrences
word_counts = pd.Series(all_text.split()).value_counts()

In [123]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
#stops = set(stopwords.words('english'))
cachedStopWords = stopwords.words("english")
extra_stop_words = ['a', 'an', 'in', 'on']
cachedStopWords += extra_stop_words

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [124]:
for stop_word in cachedStopWords:
    word_counts = word_counts.drop(stop_word, errors='ignore')

In [125]:
word_counts.to_excel('word_counts.xlsx', index=True)

In [130]:
import pandas as pd
import spacy

languages = pd.read_csv('languages.csv')
platforms = pd.read_csv('platforms.csv')
databases = pd.read_csv('databases.csv')
frameworks_tools = pd.read_csv('frameworks_tools_etc.csv')

patterns = []
for x in languages.name.tolist():
    patterns.append({"label": "PROG_LANG", "pattern": [{"lower": w.lower()} for w in str(x).split()],"id": "SKILLS"})

for x in databases.name.tolist():
    patterns.append({"label": "DB","pattern": [{"lower": w.lower()} for w in str(x).split()], "id": "SKILLS"})

for x in platforms.name.tolist():
    patterns.append({"label": "PLATFORM", "pattern": [{"lower": w.lower()} for w in str(x).split()], "id": "SKILLS"})

for x in frameworks_tools.name.tolist():
    patterns.append({"label": "FRAMEWORKS", "pattern": [{"lower": w.lower()} for w in str(x).split()], "id": "SKILLS"})

nlp = spacy.load("en_core_web_sm")
ruler = nlp.add_pipe("entity_ruler", before="ner")
ruler.add_patterns(patterns)
ruler.to_disk("patterns.jsonl")

In [132]:
import spacy
nlp = spacy.load("en_core_web_sm")
ruler = nlp.add_pipe("entity_ruler", before="ner")
patterns = "patterns.jsonl"
ruler.from_disk(patterns)

In [133]:
text = """
Semrush
is a leading digital marketing toolkit for SEO, PPC, and content marketing professionals worldwide, with 7 million marketers having already used Semrush to improve online marketing activities, monitor their website’s performance, as well as their competitors’.
With over 45 tools within the software and search data for 140 countries, Semrush provides solutions for in-house marketing teams and agencies working with clients in any industry. Semrush is an absolute leader in helping good marketers become great!
The Semrush
Enterprise solutions unit
is building new innovative solutions for large companies to unlock growth by answering recurring, complex, and unknown Digital Marketing-related questions (semi) automatically. This includes the collection, storage, and computation of huge amounts of data to create insights that are otherwise hard or impossible to get. Our solutions will connect data using machine learning to remove dependencies and friction for SEO, product, engineering, and marketing teams. New visually storytelling-like reports will help companies make better decisions about their Digital Marketing budgets.
Joining our new unit means you will be working with us on the largest Digital Marketing data set available, and you will be part of shaping the Digital Marketing world with new solutions that are transforming how large enterprise companies do Digital Marketing. The teams will include product management, engineering, data science, product design, customer success, and marketing roles.
For the new unit, the office will be in Berlin, but remote work (partially) is possible too.
Key responsibilities
Use analytical and statistical methods to analyze large amounts of digital marketing data, extracting actionable insights using advanced techniques such as data analysis, data mining, optimization tools, and machine learning.
Develop predictive models based on historical data that provide future predictions of customer website performance.
Validate and refine model assumptions with statistical analysis and testing.
Build production-ready services that can be consumed by product teams.
Technologies
Python
R
Tydverse
SQL
PyTorch, TensorFlow, NumPy, Pandas, Keras, JAX, Julia
postgreSQL, MySQL, Snowflake, GCP/AWS
Java
Job requirements
3+ years experience with applied Machine/Deep Learning.
Understanding of data structures, data modeling, and software architecture.
Strong coding experience and proven ability to implement ML packages and neural network architectures using frameworks such as PyTorch or Tensorflow.
Great communication skills and an ability to translate business context into data-oriented hypotheses to drive impact.
BSc or BA degree in computer science, data science, applied math, or a related field.
An appreciation for the connection between the product experience and models you build.
Benefits
This offer stands for the “FLEX” work format: some days a week you work from the office, and some #wfh. Trust us, it’s an efficient way of work. Proven!
Flexible working day start that would suit a night owl and an early bird alike. You can start between 9:00 am and 12:00 pm.
Private health insurance program. Life insurance.
Agile approach to work (we’ll gladly teach you).
Training/online courses and workshops/conferences/books to improve your hard and soft skills.
Financial compensation for sports and hobbies (gym/dancing/languages/horseback riding/painting/wakeboard… and many more. It’s really up to you!)
Corporate psychologist consultations. Mental health is as important as physical well-being.
Awesome parties, team building, and corporate events in different formats – both online and offline, depending on the current global situation. We can have fun in any case!
Semrush is an equal opportunity employer. We celebrate diversity and are committed to creating an inclusive environment for all employees. We do not discriminate based on race, religion, creed, color, national origin, sex, pregnancy, sexual orientation, gender identity, gender expression, age, ancestry, physical or mental disability, or medical condition, including medical characteristics, genetic identity, marital status, military service, or any other classification protected by applicable local, state or federal laws. All employment decisions are based on business needs, job requirements, merit, and individual qualifications.
"""
doc = nlp(text)
doc
ents = [{'label': entity.label_, 'text': entity.text} for entity in doc.ents if entity.ent_id_ == 'SKILLS']
ents

[{'label': 'PROG_LANG', 'text': 'Python'},
 {'label': 'PROG_LANG', 'text': 'R'},
 {'label': 'FRAMEWORKS', 'text': 'PyTorch'},
 {'label': 'FRAMEWORKS', 'text': 'TensorFlow'},
 {'label': 'FRAMEWORKS', 'text': 'NumPy'},
 {'label': 'FRAMEWORKS', 'text': 'Pandas'},
 {'label': 'FRAMEWORKS', 'text': 'Keras'},
 {'label': 'PROG_LANG', 'text': 'Julia'},
 {'label': 'DB', 'text': 'postgreSQL'},
 {'label': 'DB', 'text': 'MySQL'},
 {'label': 'DB', 'text': 'Snowflake'},
 {'label': 'PLATFORM', 'text': 'GCP'},
 {'label': 'PLATFORM', 'text': 'AWS'},
 {'label': 'FRAMEWORKS', 'text': 'PyTorch'},
 {'label': 'FRAMEWORKS', 'text': 'Tensorflow'}]

In [148]:
def extract_tech(description):
    doc = nlp(description)
    frameworks, databases, platforms, prog_langs = [], [], [], []

    for entity in doc.ents:
        if entity.ent_id_ == 'SKILLS':
            if entity.label_ == 'PROG_LANG' and entity.text not in prog_langs:
                prog_langs.append(entity.text)
            if entity.label_ == 'PLATFORM' and entity.text not in platforms:
                platforms.append(entity.text)
            if entity.label_ == 'DB' and entity.text not in databases:
                databases.append(entity.text)
            if entity.label_ == 'FRAMEWORKS' and entity.text not in frameworks:
                frameworks.append(entity.text)

    #prog_langs = " ".join(prog_langs)
    #platforms = " ".join(platforms)
    #databases = " ".join(databases)
    #frameworks = " ".join(frameworks)
    data = (prog_langs, platforms, databases, frameworks)
    data = tuple([x for x in data if x])
    if data:
        return data
    else:
        return None

In [152]:
all_data['keywords'] = all_data['Description'].apply(extract_tech)
#all_data.value_counts('Place').head(20)


In [171]:
all_data['keywords'].head()

0    ([Python], [AWS], [Redshift], [Tableau, Pandas...
1                      ([Python], [Tableau, Power BI])
2    ([Python], [Azure Data Lake], [SQL Server], [D...
3                              ([Access, SQL Server],)
4                         ([Snowflake], [Looker, ETL])
Name: keywords, dtype: object

In [227]:
teste = all_data[['Job_ID','keywords']].explode("keywords").reset_index(drop=True)
teste = teste[teste["keywords"].notna()]
teste["keywords"] = teste["keywords"].apply(lambda x: ",".join(x))
exploded_df = teste.assign(keywords=teste["keywords"].str.split(",")).explode("keywords").reset_index(drop=True)
exploded_df

,Job_ID,keywords
0,3449267566,Python
1,3449267566,AWS
2,3449267566,Redshift
3,3449267566,Tableau
4,3449267566,Pandas
...,...,...
36225,3575199096,Power BI
36226,3575199096,Tableau
36227,3561496857,R
36228,3561496857,Python


In [229]:
all_data.to_csv("jobs.csv",index=False)
exploded_df.to_csv("skills.csv",index=False)

In [286]:
exploded_df = exploded_df[exploded_df["keywords"] != "Access"]
exploded_df["keywords"].value_counts().head(10).values/len(all_data)

counts = pd.DataFrame(exploded_df["keywords"].value_counts())
counts['pct'] = counts['keywords']/len(all_data)
counts
counts.head(10)
counts = counts.head(10).iloc[::-1]
#exploded_df[,]
#px.bar(counts.head(10), x=counts.head(10).index, y=counts.head(10).values, labels={"x": "Industry", "y": "Count"})
fig = px.bar(counts.head(10), x='pct', y=counts.head(10).index,text=counts.head(10)['pct'].apply(lambda x: f'{x:,.2%}'), orientation='h')

# Customize the layout
fig.update_layout(
    title='Keyword Distribution in Job Ads',
    xaxis_title='Percentage',
    yaxis_title='Keywords',
    margin=dict(l=100, r=20, t=70, b=70),
    xaxis_tickformat=".2%",
    width=700
)
fig.update_traces(hovertemplate='%{x:,.2%}')
# Show the plot
fig.show()

In [300]:
counts.reset_index().to_csv("counts.csv",index=False)

In [296]:
counts2 = pd.read_csv("counts.csv")
counts2.rename(columns={'Unnamed: 0':'skill'})
ig = px.bar(counts2.head(10), x='pct', y=counts2.index,text=counts2['pct'].apply(lambda x: f'{x:,.2%}'), orientation='h')

# Customize the layout
fig.update_layout(
    title='Keyword Distribution in Job Ads',
    xaxis_title='Percstreamlit run appentage',
    yaxis_title='Keywords',
    margin=dict(l=100, r=20, t=70, b=70),
    xaxis_tickformat=".2%",
    width=700
)
fig.update_traces(hovertemplate='%{x:,.2%}')
fig.show()